# Curated Transactions with Merchant data

In [1]:
import os
# get the accessKey and secretKey from Environment
accessKey = os.environ['AWS_ACCESS_KEY_ID']
secretKey = os.environ['AWS_SECRET_ACCESS_KEY']

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("Jupyter")
        .master("local[*]")

        .config("spark.jars.packages",
                "org.apache.iceberg:iceberg-spark-runtime-4.0_2.13:1.10.1,"
                "org.apache.iceberg:iceberg-aws-bundle:1.10.1")

        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio-1:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.access.key", accessKey)
        .config("spark.hadoop.fs.s3a.secret.key", secretKey)
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

    
        # Iceberg catalog
        .config("spark.sql.catalog.hiverest", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.hiverest.type", "rest")
        .config("spark.sql.catalog.hiverest.uri", "http://hive-metastore:9084/iceberg")
        .config("spark.sql.catalog.hiverest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.hiverest.warehouse", "s3a://admin-bucket/iceberg/warehouse")

        # ⭐ REQUIRED FOR MINIO WITH ICEBERG AWS SDK
        .config("spark.sql.catalog.hiverest.s3.endpoint", "http://minio-1:9000")
        .config("spark.sql.catalog.hiverest.s3.path-style-access", "true")
        .config("spark.sql.catalog.hiverest.s3.access-key-id", accessKey)
        .config("spark.sql.catalog.hiverest.s3.secret-access-key", secretKey)

        .config("spark.sql.defaultCatalog", "hiverest")

        .config(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )

        .getOrCreate()
)

## Test SQL Statement using JupySQL

In [2]:
%load_ext sql
%sql spark

In [3]:
%%sql
describe payment_db.raw_transaction_t

Running query in 'SparkSession'

10 rows affected.

Field 1,Field 2,Field 3
transaction_id,string,None
card_number,string,None


In [5]:
%%sql
SELECT t.transaction_id
    , t.card_number
    , t.currency
    , t.amount
    , t.channel
    , t.transaction_date
    , CONCAT(m.name, ' ', UPPER(m.city), ' ', m.country) AS booking_text
    , m.name
    , m.country
    , m.city
    , m.category_name
FROM payment_db.raw_transaction_t t
LEFT JOIN refdata_db.raw_merchant_t m
    ON (t.merchant_id = m.merchant_id);

Running query in 'SparkSession'

++
||
++
++

## Using Spark SQL to execute SQL Join

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, upper, col

# Define the SQL query
query = """
SELECT t.transaction_id,
       t.card_number,
       t.currency,
       t.amount,
       t.channel,
       t.transaction_date,
       CONCAT(m.name, ' ', UPPER(m.city), ' ', m.country) AS booking_text,
       m.name,
       m.country,
       m.city,
       m.category_name
FROM payment_db.raw_transaction_t t
LEFT JOIN refdata_db.raw_merchant_t m
       ON t.merchant_id = m.merchant_id
"""

# Execute the query
result_df = spark.sql(query)

# Write to Iceberg table
result_df.write.format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("payment_db.cur_transaction_with_merchant_sql_t")

In [10]:
%%sql
SELECT * FROM payment_db.cur_transaction_with_merchant_sql_t

Running query in 'SparkSession'

++
||
++
++

## Using PySpark to execute SELECT with JOIN

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, upper, col

# Read source tables
transactions_df = spark.table("payment_db.raw_transaction_t")
merchants_df = spark.table("refdata_db.raw_merchant_t")

# Join and compute new columns
result_df = transactions_df.alias("t") \
    .join(
        merchants_df.alias("m"),
        col("t.merchant_id") == col("m.merchant_id"),
        "left"
    ) \
    .select(
        col("t.transaction_id"),
        col("t.card_number"),
        col("t.currency"),
        col("t.amount"),
        col("t.channel"),
        col("t.transaction_date"),
        concat_ws(" ", col("m.name"), upper(col("m.city")), col("m.country")).alias("booking_text"),
        col("m.name"),
        col("m.country"),
        col("m.city"),
        col("m.category_name")
    )

# Write to Iceberg table
result_df.write.format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("payment_db.cur_transaction_with_merchant_t")

In [14]:
%%sql
SELECT * FROM payment_db.cur_transaction_with_merchant_t

Running query in 'SparkSession'

++
||
++
++